<h1>Opacus : Train PyTorch models with Differential Privacy </h1>
Opacus is a pytorch-based framework,it was developped by Meta AI to allow us to train our models with differential privacy, and this is by transforming any SGD based optimization algorithm to its variant DP-SGD, the idea of DP-SGD is adding gaussian noise and performing gradient clipping to the gradients and using the poisson batch selection shceme, Opacus provides also the tracking the DP budget during the training.
Note that Opacus allow us to make only local differential privacy which is useful for defending against the distributed backdoor attack, and in order to perform a global differential privacy architecture, we have to make some modifications on the strategy and add gaussian noises to the model updates before aggregating them.<br>

<h3>Ressources</h3>
<ul>
  <li> <a href="https://youtu.be/U1mszp8lzUI"> An intro video </a></li>
<li> <a href="https://opacus.ai/"> Opacus website</a></li>
<li> <a href="http://dimacs.rutgers.edu/~graham/pubs/papers/opacus.pdf"> Opacus research paper</a></li>
</ul>

### Installing dependencies

First, we install the necessary packages:

In [1]:
!pip install -q flwr[simulation] torch torchvision opacus



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.9/216.9 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 KB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.5/90.5 KB 353.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 KB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 KB 37.5 MB/s eta 

Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple
import warnings
import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision.models import vgg11,vgg16,resnet18
warnings.filterwarnings("ignore", category=UserWarning)
warnings.simplefilter("ignore")
DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.1+cu116 and Flower 1.3.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda:0`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [3]:


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=16)
    return trainloaders, valloaders, testloader



In [4]:
from opacus import PrivacyEngine
MAX_GRAD_NORM = 1.
EPSILON = 20.0
DELTA = 1e-5
EPOCHS = 6


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [5]:
import numpy as np
from opacus.utils.batch_memory_manager import BatchMemoryManager

class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x )  :
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)





MAX_PHYSICAL_BATCH_SIZE = 32


def train(model, train_loader, optimizer, privacy_engine,epoch, device):
    model.train()
    criterion = nn.CrossEntropyLoss()

    losses = []
    top1_acc = []
    
    with BatchMemoryManager(
        data_loader=train_loader, 
        max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
        optimizer=optimizer
    ) as memory_safe_data_loader:

        for i, (images, target) in enumerate(memory_safe_data_loader): 
            optimizer.zero_grad()
            images = images.to(device)
            target = target.to(device)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            preds = np.argmax(output.detach().cpu().numpy(), axis=1)
            labels = target.detach().cpu().numpy()

            # measure accuracy and record loss
            acc = (preds == labels).mean()

            losses.append(loss.item())
            top1_acc.append(acc)

            loss.backward()
            optimizer.step()
            if (i+1) % 200 == 0:
                          epsilon = privacy_engine.get_epsilon(DELTA)
                          print(
                              f"\tTrain Epoch: {epoch} \t"
                              f"Loss: {np.mean(losses):.6f} "
                              f"Acc@1: {np.mean(top1_acc) * 100:.6f} "
                              f"(ε = {epsilon:.2f}, δ = {DELTA})"
                          )
                  
    
    



def test(net, testloader,device:str):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    #net.to("cpu")
    return loss, accuracy



<h2>NN layers and DP</h2>
Certain nn.Modules are not compatible with differential privacy! One of them is the popular BatchNorm layer (the reason being that it would require to make even the forward loop privacy-aware, because now each sample's feature depends on its peers in a batch). While we could do it, we don't think BatchNorm is worth the hassle.

Opacus comes with a utility to convert BatchNorm models to GroupNorm, which does not have this issue (you can of course also convert them to InstanceNorm, or LayerNorm but let's stick to GroupNorm since it's a generalization of those two anyway).


source : <a href="https://colab.research.google.com/drive18GRMyixn43T_lHF6ZheoAJn4-J8L3LAL#scrollTo=1f6H4PI7Ndgb" >Opacus demo</a>

We can check the validity of our model, and if it contains some incompatible layers we can then fix them using the validator function

In [6]:
from opacus.validators import ModuleValidator

errors = ModuleValidator.validate(Net(), strict=False)
errors


[]

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [7]:
warnings.simplefilter("ignore")
class FlowerClientDP(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.device = torch.device(f"cuda:{int(self.cid)%4}" if torch.cuda.is_available() else "cpu")
        self.net.to(self.device)
        self.privacy_engine  = PrivacyEngine(secure_mode=False)
        self.optimizer = torch.optim.SGD(self.net.parameters(), lr = 0.001)
        self.net, self.optimizer, self.trainloader = self.privacy_engine.make_private_with_epsilon(
                                                      module=self.net,
                                                      optimizer=self.optimizer,
                                                      data_loader=self.trainloader,
                                                      epochs=1,
                                                      target_epsilon=EPSILON,
                                                      target_delta=DELTA,
                                                      max_grad_norm=MAX_GRAD_NORM,
                                                  )   

        #change net.params here
        
       
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        params = get_parameters(self.net)
        
        return params

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
     
        for _ in range(EPOCHS):
          train(self.net, self.trainloader, self.optimizer ,self.privacy_engine,epoch=_,device = self.device)
        epsilon = self.privacy_engine.get_epsilon(DELTA)
        print(f"epsilon of client {self.cid} : eps = {epsilon}")
        return get_parameters(self.net), len(self.trainloader), {"epsilon" : epsilon}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader,device = self.device)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}



def client_fn(cid) -> FlowerClientDP:
    device = torch.device(f"cuda:{int(cid)%4}" if torch.cuda.is_available() else "cpu")
    net = Net()
    net = net.to(device)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    
    

    return FlowerClientDP(cid, net, trainloader, valloader)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features. 

## Server-side parameter **evaluation**





In [8]:
NUM_CLIENTS = 10
trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


In [9]:
warnings.simplefilter("ignore")
net = Net()
net.to(DEVICE)
privacy_engine = PrivacyEngine(secure_mode=False)
optimizer = torch.optim.RMSprop(net.parameters(), lr=0.01)

net, optimizer, trainloader = privacy_engine.make_private_with_epsilon(
                                                      module=net,
                                                      optimizer=optimizer,
                                                      data_loader=trainloaders[0],
                                                      epochs=1,
                                                      target_epsilon=EPSILON,
                                                      target_delta=DELTA,
                                                      max_grad_norm=MAX_GRAD_NORM,
                                                  )


In [10]:
# The `evaluate` function will be by Flower called after every round

global_model = Net() 

global_model = global_model.to(DEVICE) 
params = get_parameters(global_model)
def evaluate(
    server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    valloader = valloaders[0]
    set_parameters(global_model, parameters)  # Update model with the latest parameters
    loss, accuracy = test(global_model, valloader,DEVICE)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [11]:

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.7,  
    fraction_evaluate=0.7,  
    min_fit_clients=4,
    min_evaluate_clients=4,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),  # 10 rounds
    strategy=strategy,
)

INFO flwr 2023-02-16 12:46:57,525 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-02-16 12:46:59,346	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
INFO flwr 2023-02-16 12:47:00,688 | app.py:179 | Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 3992779161.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7985558324.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'GPU': 1.0, 'object_store_memory': 3992779161.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0, 'memory': 7985558324.0}
INFO flwr 2023-02-16 12:47:00,697 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-02-16 12:47:00,704 | server.py:266 | Using initial parameters provided by strategy
INFO:flwr:Using initial parameters provided by strategy
INFO 

Server-side evaluation loss 0.07371957349777222 / accuracy 0.1


(launch_and_fit pid=624) /usr/local/lib/python3.8/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
(launch_and_fit pid=624)   warnings.warn(
(launch_and_fit pid=624) /usr/local/lib/python3.8/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
(launch_and_fit pid=624)   warnings.warn(
(launch_and_fit pid=624) /usr/local/lib/python3.8/dist-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
(launch_and_fit pid=624)   z = np.log((np.exp(t) + q - 1) / q)
(launch_and_fit pid=625) /usr/local/lib/python3.8/dist-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is 

(launch_and_fit pid=624) [Client 2] fit, config: {}


(launch_and_fit pid=625) /usr/local/lib/python3.8/dist-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
(launch_and_fit pid=625)   warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


...

(launch_and_fit pid=624) epsilon of client 6 : eps = 38.57552953833644
(launch_and_fit pid=625) [Client 5] fit, config: {}
(launch_and_fit pid=624) [Client 8] fit, config: {}
(launch_and_fit pid=625) 	Train Epoch: 0 	Loss: 2.279129 Acc@1: 15.414760 (ε = 19.82, δ = 1e-05)
(launch_and_fit pid=624) 	Train Epoch: 0 	Loss: 2.278905 Acc@1: 15.776152 (ε = 20.00, δ = 1e-05)
(launch_and_fit pid=625) 	Train Epoch: 1 	Loss: 2.279213 Acc@1: 15.053044 (ε = 24.84, δ = 1e-05)
(launch_and_fit pid=624) 	Train Epoch: 1 	Loss: 2.277974 Acc@1: 16.287857 (ε = 24.81, δ = 1e-05)
(launch_and_fit pid=624) 	Train Epoch: 2 	Loss: 2.276237 Acc@1: 16.169143 (ε = 28.99, δ = 1e-05)
(launch_and_fit pid=624) 	Train Epoch: 3 	Loss: 2.276867 Acc@1: 16.492169 (ε = 32.40, δ = 1e-05)
(launch_and_fit pid=625) 	Train Epoch: 4 	Loss: 2.276331 Acc@1: 15.140436 (ε = 35.58, δ = 1e-05)
(launch_and_fit pid=624) 	Train Epoch: 4 	Loss: 2.274216 Acc@1: 17.254428 (ε = 35.41, δ = 1e-05)
(launch_and_fit pid=625) 	Train Epoch: 5 	Loss: 2

DEBUG flwr 2023-02-16 13:49:22,385 | server.py:229 | fit_round 10 received 7 results and 0 failures
DEBUG:flwr:fit_round 10 received 7 results and 0 failures
INFO flwr 2023-02-16 13:49:22,580 | server.py:116 | fit progress: (10, 0.07300526762008668, {'accuracy': 0.142}, 3741.4687943930003)
INFO:flwr:fit progress: (10, 0.07300526762008668, {'accuracy': 0.142}, 3741.4687943930003)
DEBUG flwr 2023-02-16 13:49:22,589 | server.py:165 | evaluate_round 10: strategy sampled 7 clients (out of 10)
DEBUG:flwr:evaluate_round 10: strategy sampled 7 clients (out of 10)


(launch_and_fit pid=625) epsilon of client 0 : eps = 38.57552953833644
Server-side evaluation loss 0.07300526762008668 / accuracy 0.142
(launch_and_evaluate pid=625) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=624) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=624) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=625) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=624) [Client 7] evaluate, config: {}
(launch_and_evaluate pid=625) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=624) [Client 5] evaluate, config: {}


DEBUG flwr 2023-02-16 13:50:33,511 | server.py:179 | evaluate_round 10 received 7 results and 0 failures
DEBUG:flwr:evaluate_round 10 received 7 results and 0 failures
INFO flwr 2023-02-16 13:50:33,513 | server.py:144 | FL finished in 3812.4013537640003
INFO:flwr:FL finished in 3812.4013537640003
INFO flwr 2023-02-16 13:50:33,515 | app.py:202 | app_fit: losses_distributed [(1, 0.07363850232533045), (2, 0.07362863656452724), (3, 0.07357161671774727), (4, 0.07351281016213553), (5, 0.07343416316168648), (6, 0.07337530054364885), (7, 0.07328757006781442), (8, 0.0731219014440264), (9, 0.07303135408673968), (10, 0.07280791848046439)]
INFO:flwr:app_fit: losses_distributed [(1, 0.07363850232533045), (2, 0.07362863656452724), (3, 0.07357161671774727), (4, 0.07351281016213553), (5, 0.07343416316168648), (6, 0.07337530054364885), (7, 0.07328757006781442), (8, 0.0731219014440264), (9, 0.07303135408673968), (10, 0.07280791848046439)]
INFO flwr 2023-02-16 13:50:33,517 | app.py:203 | app_fit: metrics

History (loss, distributed):
	round 1: 0.07363850232533045
	round 2: 0.07362863656452724
	round 3: 0.07357161671774727
	round 4: 0.07351281016213553
	round 5: 0.07343416316168648
	round 6: 0.07337530054364885
	round 7: 0.07328757006781442
	round 8: 0.0731219014440264
	round 9: 0.07303135408673968
	round 10: 0.07280791848046439
History (loss, centralized):
	round 0: 0.07371957349777222
	round 1: 0.07369145441055298
	round 2: 0.07366289901733399
	round 3: 0.07362965869903565
	round 4: 0.07358869695663452
	round 5: 0.07353930377960205
	round 6: 0.07347536087036133
	round 7: 0.0733940806388855
	round 8: 0.07329087018966675
	round 9: 0.07316101360321045
	round 10: 0.07300526762008668
History (metrics, centralized):
{'accuracy': [(0, 0.1), (1, 0.1), (2, 0.1), (3, 0.1), (4, 0.1), (5, 0.1), (6, 0.1), (7, 0.112), (8, 0.128), (9, 0.13), (10, 0.142)]}

In [13]:
#test the global model after the federated training
loss, acc = test(global_model,testloader,DEVICE)
print(f'loss = {loss}')
print(f'accuracy = {acc}')

loss = 0.14202568185329437
accuracy = 0.1646
